In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from typing import Tuple
from price_impact.modelling.Futures import FutureContract, Spot
from price_impact.modelling.backtest_analysis import BacktestSettings, BacktestResults
from price_impact.execution.discrete_execution.impact_models import  PropagatorImpactModel


import datetime as dt
from pathlib import Path
np.random.seed(0)

PLOT_FOLDER = Path("./plots")
plt.style.use('bmh')

In [ ]:
backtest_settings = BacktestSettings(
        train_len=6,
        test_len=3,
        bin_freq='10s',
        forecast_horizon_s=10,
        smoothing_str='sma20',
        decay=3600)

In [ ]:
freq_m = 6

lam = 1
trading_hours = 10

do_bounds = False
total_volume = 0.1
half_life_s = 60*60

stdev = 1
spread_bps = 0
options = {'maxiter': 1e6}
fig, ax = plt.subplots()

colors = ['#1f77b4', '#ff7f0e']
for delta, color in zip([1, 0.975], colors):
    pim = PropagatorImpactModel(half_life_s=half_life_s, delta=delta,
                                trading_hours=trading_hours, freq_m=freq_m, stdev=stdev,
                                lambdas=lam)

    res = pim.get_optimal_volume(spread_bps=spread_bps, total_volume=total_volume,
                                 do_bounds=do_bounds, tol=1e-14, divisor=1, options=options)
    ax.bar(range(len(res.x)), res.x, alpha=0.5, label=f"delta = {delta}", color=color)
ax.legend()

fig = ax.get_figure()

fig.suptitle(f"Optimal Execution under DPM. Bins: {len(res.x)}. X={total_volume}")
# fig.tight_layout()
ax.legend(loc=3)
ax.set_ylabel("Volume")
ax.set_xlabel("Time bin")
fig.savefig(PLOT_FOLDER/'optimal_execution'/f'oe_weak_nonlinear.png')

In [ ]:
spot = Spot('NQ')

backtest_results = BacktestResults(spot, backtest_settings)

results_df = backtest_results.results_df
del_lams = results_df.query("variable=='coef' and test_start=='2017-01-01'").pivot(index='impact',columns='hour',values='value').mean(axis=1)

In [ ]:
freq_m = 15
trading_hours = 23

do_bounds = False
total_volume = 0.1
half_life_s = 60*60

stdev = 0.0002
spread_bps = 0
options = {'maxiter':1e4}

fig, axs = plt.subplots(2, 2, figsize=(10,6))

for idx, delta in enumerate([0.7, 0.5, 0.3, 0.1]):
    lam = del_lams[delta]

    i, j = idx//2, idx % 2
    ax = axs[i][j]
    
    pim = PropagatorImpactModel(half_life_s=half_life_s, delta=delta,
                                trading_hours=trading_hours, freq_m=freq_m, stdev=stdev,
                                lambdas=lam)

    res = pim.get_optimal_volume(spread_bps=spread_bps, total_volume=total_volume,
                                 do_bounds=do_bounds, tol=1e-9, divisor=1, options=options)

    ax.bar(range(len(res.x)), res.x)
    ax.set_title(f'$\delta={delta}, \lambda={lam:.2f}$')
    
        
    if i ==1:
        ax.set_xlabel('Time Bin')
    if not j:
        ax.set_ylabel('Volume')

fig.suptitle(f"Optimal volume schedule for different ($\lambda, \delta $)")
fig.tight_layout()
fig.savefig(PLOT_FOLDER/'optimal_execution'/'oe_effect_of_delta.png')

In [ ]:
freq_m = 15

neg_trades = dict()

delta = 0.5
lam = del_lams[delta]

trading_hours = 23
do_bounds = False
total_volume = 0.1
half_life_s = 60*60

stdev = 0.0002

fig, axs = plt.subplots(2, 3, figsize=(10,6))

for idx, spread_bps in enumerate([0,10,25,50,100]):
    lam = del_lams[delta]


    i, j = idx//3, idx % 3
    ax = axs[i][j]
    
    pim = PropagatorImpactModel(half_life_s=half_life_s, delta=delta,
                                trading_hours=trading_hours, freq_m=freq_m, stdev=stdev,
                                lambdas=lam)

    res = pim.get_optimal_volume(spread_bps=spread_bps, total_volume=total_volume,
                                 do_bounds=do_bounds, tol=1e-9, divisor=1)

    ax.bar(range(len(res.x)), res.x)
    ax.set_title(f'Spread {spread_bps}bps', fontsize=10)

    neg_trades[spread_bps] = res.x[res.x<0].sum()
    
    if i ==1:
        ax.set_xlabel('Time Bin')
    if not j:
        ax.set_ylabel('Volume')

spread_bps=0
do_bounds=True

pim = PropagatorImpactModel(half_life_s=half_life_s, delta=delta, trading_hours=trading_hours,
                            freq_m=freq_m, stdev=stdev, lambdas=lam)
res = pim.get_optimal_volume(spread_bps=spread_bps, total_volume=total_volume,
                                 do_bounds=do_bounds, tol=1e-9, divisor=1)
ax = axs[-1][-1]
ax.bar(range(len(res.x)), res.x)
ax.set_title(f'Spread {spread_bps}bps. Bounds = {do_bounds}', fontsize=10)
ax.set_xlabel('Time Bin')


fig.suptitle(f"Effect of adding spread cost. $\delta={delta}, \lambda={lam:.2f}$.")
fig.tight_layout()

fig.savefig(PLOT_FOLDER/'optimal_execution'/'oe_effect_of_spread.png')

In [ ]:
neg_trades_df  = pd.Series(neg_trades).to_frame().reset_index()
neg_trades_df.columns = ['Spread (bps)', 'Negative Volume']

neg_trades_df.to_latex(f'./tables/optimal_execution/effect_of_spread.tex', index=False)
neg_trades_df

In [ ]:
fig, axs = plt.subplots(2, 3, figsize=(10,6))

delta = 0.5
lam = del_lams[delta]

spread_bps = 0
do_bounds = False
half_life_s = 60*60

trading_hours = 23
total_volume = 0.1
stdev = 0.0002

for idx, freq_m in enumerate([60, 45, 30, 20, 10, 5]):

    i, j = idx//3, idx % 3
    ax = axs[i][j]
    
    pim = PropagatorImpactModel(half_life_s=half_life_s, delta=delta,
                                trading_hours=trading_hours, freq_m=freq_m, stdev=stdev,
                                lambdas=lam)

    res = pim.get_optimal_volume(spread_bps=spread_bps, total_volume=total_volume,
                                 do_bounds=do_bounds, tol=1e-9, divisor=1)
                                 

    ax.bar(range(len(res.x)), res.x)
    ax.set_title(f'Frequency {freq_m}m', fontsize=10)

    if i ==1:
        ax.set_xlabel('Time Bin')
    if not j:
        ax.set_ylabel('Volume')
        
fig.suptitle(f"Effect of trading frequency. $\delta={delta}, \lambda={lam:.2f}$.")
fig.tight_layout()
fig.savefig(PLOT_FOLDER/'optimal_execution'/'oe_effect_of_freq.png')

### Tol

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(10,6))

delta = 0.5
lam = del_lams[delta]

spread_bps = 0
do_bounds = False
half_life_s = 60*60

trading_hours = 23
total_volume = 0.1
stdev = 0.0002

freq_m = 15

for idx, tol_exp in enumerate([4, 5, 6, 8]):
    
    tol = 1/(10**tol_exp)
    print(tol)
    i, j = idx//2, idx % 2
    ax = axs[i][j]
    
    pim = PropagatorImpactModel(half_life_s=half_life_s, delta=delta,
                                trading_hours=trading_hours, freq_m=freq_m, stdev=stdev,
                                lambdas=lam)

    res = pim.get_optimal_volume(spread_bps=spread_bps, total_volume=total_volume,
                                 do_bounds=do_bounds, tol=tol, divisor=1)

    ax.bar(range(len(res.x)), res.x)
    ax.set_title(f'tol = 1e-{tol_exp}', fontsize=10)
    
    
    if i ==1:
        ax.set_xlabel('Time Bin')
    if not j:
        ax.set_ylabel('Volume')
        

fig.suptitle(f"Effect of tolerence parameter. $\delta={delta}, \lambda={lam:.2f}$.")
fig.tight_layout()
fig.savefig(PLOT_FOLDER/'optimal_execution'/'oe_effect_of_tol.png')